# Desafio 2
Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

Probar términos de interés y explicar similitudes en el espacio de embeddings.

Intentar plantear y probar tests de analogías.

Graficar los embeddings resultantes.

Sacar conclusiones.

Instala la librería datasets de Hugging Face para acceder a conjuntos de datos.

In [68]:
pip install datasets

Importa las librerías necesarias para cargar datos, crear modelos de Word2Vec, y visualizar resultados.

In [69]:
from datasets import load_dataset
from gensim.models import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

Carga el dataset SNLI

In [70]:
# Cargar el dataset GLUE subset sst-2
dataset = load_dataset("stanfordnlp/snli")
df= pd.DataFrame(dataset['train'])

In [72]:
len( df )

550152

In [73]:
df = pd.DataFrame(dataset['train']['premise'], columns=["premise"])
df.drop_duplicates(inplace=True)
df.shape

(150736, 1)

##Preprocesamiento

In [74]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-74-99a399a8b871>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [75]:
# Demos un vistazo
sentence_tokens[:2]

[['a',
  'person',
  'on',
  'a',
  'horse',
  'jumps',
  'over',
  'a',
  'broken',
  'down',
  'airplane'],
 ['children', 'smiling', 'and', 'waving', 'at', 'camera']]

##Crear los vectores (word2vvc)

In [76]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [77]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)

In [78]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [79]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 150736


In [80]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 7554


## Entrenamiento de embeddings

In [81]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 8633490.0
Loss after epoch 1: 7168520.0
Loss after epoch 2: 6794294.0
Loss after epoch 3: 6657308.0
Loss after epoch 4: 6606584.0
Loss after epoch 5: 6577784.0
Loss after epoch 6: 6495608.0
Loss after epoch 7: 6438184.0
Loss after epoch 8: 6368996.0
Loss after epoch 9: 5628488.0
Loss after epoch 10: 1693424.0
Loss after epoch 11: 1648448.0
Loss after epoch 12: 1608728.0
Loss after epoch 13: 1571512.0
Loss after epoch 14: 1533024.0
Loss after epoch 15: 1491136.0
Loss after epoch 16: 1458920.0
Loss after epoch 17: 1415912.0
Loss after epoch 18: 1381640.0
Loss after epoch 19: 1352392.0


(23081213, 36931460)

## Ensayar
### Prueba de analogias

In [82]:
# Palabras que MÁS se relacionan con...:
print(w2v_model.wv.most_similar(positive=['horse', 'jump'], negative=['person'], topn=5))

[('leap', 0.38963237404823303), ('horseback', 0.37196215987205505), ('horses', 0.3712778389453888), ('hurdle', 0.3706740140914917), ('stride', 0.36630383133888245)]


Encuentra las palabras que están contextualmente más relacionadas con 'horse' y 'jump' al quitar 'person'. Esto refleja cómo el modelo entiende las asociaciones semánticas basadas en el corpus.

In [83]:
print(w2v_model.wv.most_similar(positive=['smiling', 'camera'], negative=['crazy'], topn=5))

[('smiles', 0.4003285765647888), ('caring', 0.35034629702568054), ('camcorder', 0.34751465916633606), ('gesticulates', 0.33764874935150146), ('takeout', 0.33502835035324097)]


El modelo muestra las palabras más relacionadas con 'smiling' y 'camera' excluyendo 'crazy', lo que proporciona una visión de las similitudes de contexto.

## Ensayar

In [84]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["peruse"], topn=10)

[('browse', 0.7479051947593689),
 ('flee', 0.728155791759491),
 ("farmer's", 0.7123233079910278),
 ('available', 0.7020047307014465),
 ('flea', 0.6824276447296143),
 ('spices', 0.6749407052993774),
 ('lettuce', 0.6705610752105713),
 ('selecting', 0.6682788729667664),
 ('browsing', 0.6592974662780762),
 ('seller', 0.652169942855835)]

Muestra las palabras que tienen una relación más fuerte con 'peruse' según el modelo entrenado.

In [85]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["horse"], topn=10)

[('use', 0.04635145887732506),
 ('measuring', 0.04579503461718559),
 ('serving', 0.015008862130343914),
 ('reviewing', 0.014689343981444836),
 ('multiple', 0.01393023319542408),
 ('washing', 0.002902029547840357),
 ('temple', 0.0004069445712957531),
 ('dim', -0.0005985419265925884),
 ('robes', -0.0025097825564444065),
 ('glass', -0.0067339385859668255)]

Encuentra las palabras que están menos relacionadas con 'horse'. Esto puede ayudar a identificar palabras en un contexto opuesto.

In [86]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["camera"], topn=10)

[('camcorder', 0.524447500705719),
 ('broadly', 0.49047237634658813),
 ('viewer', 0.4794682562351227),
 ('sights', 0.4606354534626007),
 ('pensive', 0.45903900265693665),
 ("camera's", 0.45532867312431335),
 ('voice', 0.4451206624507904),
 ('michael', 0.4411706030368805),
 ('booklet', 0.4377310574054718),
 ('curiously', 0.434591680765152)]

Muestra las palabras más cercanas en contexto a 'camera'.

In [87]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["smiling"], topn=10)

[('cinder', 0.017938319593667984),
 ('dry', 0.016183335334062576),
 ('glider', 0.012503043748438358),
 ('way', 0.009737120941281319),
 ('biker', 0.009433901868760586),
 ('form', 0.00879032164812088),
 ('series', 0.0028385629411786795),
 ('jets', -0.0006394082447513938),
 ('complete', -0.0019747684709727764),
 ('engaged', -0.003408650867640972)]

Encuentra palabras menos asociadas a 'smiling' en el espacio de embeddings.

In [88]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["personitas"])

KeyError: "Key 'personitas' not present in vocabulary"

Intenta encontrar palabras similares usando una palabra fuera del vocabulario, lo que puede generar errores o resultados vacíos.

In [89]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("smiling")
print(vector_love)

[-0.19463693 -0.13363379  0.24833597 -0.25546852 -0.34012341 -0.49376348
 -0.21987657  0.7508575  -0.2017214  -0.01726481  0.33083588 -0.15334775
 -0.21345696  0.29398057 -0.25296152  0.35275754 -0.17929032  0.07357505
 -0.09261425  0.09505588  0.33308882 -0.17235437  0.0936766  -0.01048534
  0.2313013  -0.49826455 -0.01240153 -0.2335476   0.1644789  -0.26676977
 -0.49975023  0.13829052 -0.01153416 -0.17409554  0.02811989  0.34137484
  0.4274253  -0.05360426 -0.18282875  0.11406602 -0.21219207 -0.14062618
  0.23529898  0.32691878  0.6096664  -0.16201793 -0.21921638  0.15018469
 -0.12900235  0.27311724 -0.37904975 -0.17950614  0.44004977 -0.11958599
  0.08537972  0.26369038 -0.107324    0.00852096  0.1071149   0.2188916
  0.10096107  0.27153903  0.10569486  0.4192863  -0.12116634  0.46034393
 -0.31794244 -0.06429811 -0.3341793  -0.10204653 -0.08209068  0.16445595
 -0.09221222 -0.11624147  0.17034437  0.03856412 -0.04710691 -0.08040558
 -0.64567465  0.08753073  0.08150253 -0.33568144  0.

Extrae el vector asociado a la palabra 'smiling', lo que permite ver su representación numérica.

In [90]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('smiling', 1.0),
 ('smiles', 0.5130669474601746),
 ('laughing', 0.47333085536956787),
 ('yawns', 0.44452354311943054),
 ('giggling', 0.4311577379703522),
 ('grins', 0.42983126640319824),
 ('ladle', 0.41926735639572144),
 ('grinning', 0.40386784076690674),
 ('frowning', 0.4018012285232544),
 ('gesticulates', 0.39961856603622437)]

Encuentra palabras similares usando un vector en lugar de una palabra, mostrando la flexibilidad del método.

## Visualizacion de agrupacion de vectores

In [91]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [92]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [93]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)